In [31]:
import datetime
import re
import isodate
import openai
import speech_recognition as sr
from googleapiclient.discovery import build
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from secrete import OPENAI_API_KEY, YOUTUBE_API_KEY


In [32]:
API_KEY = YOUTUBE_API_KEY
openai.api_key = OPENAI_API_KEY
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

genai.configure()

In [33]:

def analyze_titles_with_chatgpt(titles):
    
    best_index = 1  
    reason = "The title is eye-catching and highlights key details like the teams, season, and excitement level."
    return f"Here's the best video based on the title: {best_index+1}. {titles[best_index]}\nReason: {reason}"

    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=50,
        temperature=0.7
    )
    answer = response.choices[0].message.content
    return answer


def get_user_input():
    print(" How would you like to search?")
    print("1.  Speak your YouTube search query")
    print("2.  Type your YouTube search query")
    
    choice = input("Enter 1 for voice or 2 for text: ").strip()

    if choice == '1':
        query = get_voice_input()
        if query:
            print(f" You said: {query}")
            return query
        else:
            print(" Could not understand audio. Switching to text input.")
    
    query = get_text_input()
    return query


def get_text_input():
    return input(" Enter your YouTube search query: ")

def get_youtube_results(query):
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=API_KEY)

    published_after = (datetime.datetime.now() - datetime.timedelta(days=14)).isoformat("T") + "Z"

    search_response = youtube.search().list(
        q=query,
        part='snippet',
        maxResults=20,
        type='video',
        order='relevance',
        publishedAfter=published_after
    ).execute()

    video_ids = [item['id']['videoId'] for item in search_response.get('items', [])]

    if not video_ids:
        return []

    video_response = youtube.videos().list(
        part='contentDetails,snippet',
        id=','.join(video_ids)
    ).execute()

    results = []
    print(f" Found {len(video_response['items'])} videos from search")

    for item in video_response.get('items', []):
        try:
            duration = isodate.parse_duration(item['contentDetails']['duration'])
            duration_minutes = duration.total_seconds() / 60

            if 4 <= duration_minutes <= 20:
                results.append({
                    'title': item['snippet']['title'],
                    'video_id': item['id'],
                    'published_at': item['snippet']['publishedAt'],
                    'channel_title': item['snippet']['channelTitle'],
                    'duration_minutes': round(duration_minutes, 2),
                    'url': f"https://www.youtube.com/watch?v={item['id']}"
                })
        except Exception as e:
            print(f" Skipped a video due to error: {e}")

    return results



#main
query = get_user_input()

if query:
    videos = get_youtube_results(query)
    if not videos:
        print(" No videos found.")
    else:
        for idx, v in enumerate(videos):
            print(f"\n{idx+1}. 🎬 {v['title']} ({v['duration_minutes']} mins)\n📺 Channel: {v['channel_title']}\n🔗 {v['url']}")

   

        


 How would you like to search?
1.  Speak your YouTube search query
2.  Type your YouTube search query


Enter 1 for voice or 2 for text:  2
 Enter your YouTube search query:  espnfc


 Found 20 videos from search

1. 🎬 Liverpool transfer talk + Premier League top 5 race heating up 🔥 | ESPN FC (11.53 mins)
📺 Channel: ESPN FC
🔗 https://www.youtube.com/watch?v=IojL5iYT17U

2. 🎬 Who will finish top 5 in the Premier League? | ESPN FC (4.3 mins)
📺 Channel: ESPN FC
🔗 https://www.youtube.com/watch?v=xW7p1z1_arQ

3. 🎬 Lamine Yamal is the DEFINITION OF A CHAMPION 🔥 'HE WAS UNSTOPPABLE!' - Ale Moreno | ESPN FC (16.43 mins)
📺 Channel: ESPN FC
🔗 https://www.youtube.com/watch?v=twrDiv3Co3c

4. 🎬 He’s on ‘ANOTHER LEVEL’ 👀 Craig Burley lauds Lamine Yamal’s performance at age 17 | ESPN FC (12.2 mins)
📺 Channel: ESPN FC
🔗 https://www.youtube.com/watch?v=JfD2adA8z40

5. 🎬 Xabi Alonso CLOSE to being named Real Madrid's NEXT MANAGER 🤯 What will he do with Mbappe? | ESPN FC (12.32 mins)
📺 Channel: ESPN FC
🔗 https://www.youtube.com/watch?v=4trbPq97CM8

6. 🎬 FA Cup final REACTION! Have Man City become too predictable? | ESPN FC (7.42 mins)
📺 Channel: ESPN UK
🔗 https://www.youtube.com/watch

In [34]:
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


video_titles = [v['title'] for v in videos] if query and videos else [
    "How to Learn Python Quickly",
    "Python Tutorial for Beginners",
    "Advanced Python Tricks and Tips",
    "Python Projects to Practice",
    "Understanding Python Basics"
]


prompt = "YouTube video titles:\n"
for i, title in enumerate(video_titles, 1):
    prompt += f"{i}. {title}\n"
prompt += "\nPlease respond only with the number of the best video based on title quality and relevance."


inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
outputs = model.generate(
    inputs,
    max_length=inputs.shape[1] + 5,
    do_sample=False,
    temperature=0.0,
    pad_token_id=tokenizer.eos_token_id
)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\n🤖 GPT-2 Suggests:")
print(response)


match = re.search(r'\b(\d+)\b', response.strip())
if match:
    best_index = int(match.group(1)) - 1
    if 0 <= best_index < len(videos):
        best_video = videos[best_index]
        print(f"\n Best Video:")
        print(f" {best_video['title']}")
        print(f" Channel: {best_video['channel_title']}")
        print(f" {best_video['url']}")
    else:
        print(" GPT-2 suggested a number outside the video list range.")
else:
    print(" Could not extract a valid number from GPT-2's response.")


🤖 GPT-2 Suggests:
YouTube video titles:
1. Liverpool transfer talk + Premier League top 5 race heating up 🔥 | ESPN FC
2. Who will finish top 5 in the Premier League? | ESPN FC
3. Lamine Yamal is the DEFINITION OF A CHAMPION 🔥 'HE WAS UNSTOPPABLE!' - Ale Moreno | ESPN FC
4. He’s on ‘ANOTHER LEVEL’ 👀 Craig Burley lauds Lamine Yamal’s performance at age 17 | ESPN FC
5. Xabi Alonso CLOSE to being named Real Madrid's NEXT MANAGER 🤯 What will he do with Mbappe? | ESPN FC
6. FA Cup final REACTION! Have Man City become too predictable? | ESPN FC
7. What else can Lamine Yamal do to win the Ballon d’Or? | ESPN FC Extra Time
8. SACKING Ruben Amorim & Ange Postecoglou?! + Craig Burley's trip to Barcelona ✈️ | ESPN FC Extra Time
9. UNCERTAINTY with Florian Wirtz's future 👀 Manchester City enters the picture 😳 | ESPN FC
10. ELCLASICO RECAP ⚽ How does Real Madrid GET BACK on Barcelona’s level?! 👀 | ESPN FC
11. BARCELONA CELEBRATES 🎉 How another LALIGA championship was captured 👏 | ESPN FC
12. 'MONUM